In [1]:
import pandas as pd 
import numpy as np 

In [2]:
from sklearn.preprocessing import LabelEncoder

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data = pd.read_csv('DATATHON_EVENT_DATASET.csv')
test_data = pd.read_csv('DATATHON_Test.csv')

In [ ]:
data.head()

In [ ]:
# data = data.drop("Expected_Fraud", axis=1)

In [ ]:
data.dtypes

In [ ]:
data['TransactionAmount'] = - data['Final_Origin_Balance'] + data['Initial_Origin_Balance']

In [ ]:
# data = data.drop(['Initial_Origin_Balance', 'Initial_Destination_Balance'], axis=1)

In [ ]:
data.columns

In [4]:
# # Function to hash Origin_ID
# def hash_origin_id(origin_id):
#     return hashlib.sha256(origin_id.encode()).hexdigest()

# # Apply the function to the Origin_ID column
# data['Hashed_Origin_ID'] = data['Origin_ID'].apply(hash_origin_id).rank(method="dense").astype(int)
# data['Hashed_Destination_ID'] = data['Destination_ID'].apply(hash_origin_id).rank(method="dense").astype(int)

encodables = ['Origin_ID', 'Destination_ID', 'Fraud']
encoders = {}
for column in encodables:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    encoders[column] = le

In [5]:
# test_data['Fraud'] = encoders['Fraud'].transform(test_data['Fraud'])

In [ ]:
data.head()

In [ ]:
# data = data.drop(['Origin_ID', 'Destination_ID'], axis=1)

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
data['Destination_ID'].value_counts()

In [ ]:
data['Origin_ID'].value_counts()

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
(data['Initial_Origin_Balance'] - data['Final_Origin_Balance']) == (data['Final_Destination_Balance'] - data['Initial_Destination_Balance'])


In [ ]:
# data = data.drop(['Origin_ID', 'Destination_ID', 'Initial_Origin_Balance', 'Initial_Destination_Balance'], axis=1)

In [ ]:
data.head()

In [ ]:
data['Transaction_Type'].value_counts()

In [ ]:
transaction_types = data['Transaction_Type'].unique()
print(transaction_types)

In [ ]:
# No null values are found except for the Expected_fraud column
null_count = data.isnull().sum()
print(null_count)  # Returns the count of nulls

In [6]:
# Create a mapping dictionary for the similar labels
transaction_type_mapping = {
    'CASH_OUUUT': 'CASH_OUT',
    'Trans_fer': 'TRANSFER',
    'TRANSFER': 'TRANSFER',  # Keeping it the same for consistency
    'CASH_OUT': 'CASH_OUT',  # Keeping it the same
    'PAYMENT': 'PAYMENT',  # Keeping it the same
    'CASH_IN': 'CASH_IN',  # Keeping it the same
}

In [7]:
# Apply the mapping to replace inconsistent labels with correct ones
data['Transaction_Type'] = data['Transaction_Type'].replace(transaction_type_mapping)
test_data['Transaction_Type'] = test_data['Transaction_Type'].replace(transaction_type_mapping)
# Verify the changes
print(data['Transaction_Type'].value_counts())
# #Transaction amount is negative if outgoing and positive for incoming

Transaction_Type
CASH_OUT    21677
PAYMENT     17079
CASH_IN     10808
TRANSFER     8272
DEBIT         314
Name: count, dtype: int64


In [ ]:
# # Assuming your DataFrame is called 'data'

# # Step 1: Create a new column 'Transaction_Sign' based on the sign of the 'Amount'
# data['Transaction_Amount_sign'] = data['TransactionAmount'].apply(lambda x: 0 if x < 0 else 1)

# # Step 2: Group the data by 'Transaction_Type' and 'Transaction_Sign' a
# grouped_data = data.groupby(['Transaction_Type', 'Transaction_Amount_sign']).size().unstack(fill_value=0)

# # Display the grouped data
# #print(grouped_data)
# # Assuming the result is stored in 'grouped_data'
# reversed_axes = grouped_data.transpose()

# # Display the reversed axes DataFrame
# print(reversed_axes)

In [ ]:
data.head()

In [8]:
data = pd.get_dummies(data, columns=['Transaction_Type'])
test_data = pd.get_dummies(test_data, columns=['Transaction_Type'])
data.head()

,Time,Amount,Origin_ID,Initial_Origin_Balance,Final_Origin_Balance,Destination_ID,Initial_Destination_Balance,Final_Destination_Balance,Fraud,Expected_Fraud,Transaction_Type_CASH_IN,Transaction_Type_CASH_OUT,Transaction_Type_DEBIT,Transaction_Type_PAYMENT,Transaction_Type_TRANSFER
0,162,183806.32,48751,19391.00,0.00,8311,382572.19,566378.51,0,No,False,True,False,False,False
1,137,521.37,31245,0.00,0.00,50731,0.00,0.00,0,No,False,False,False,True,False
2,179,3478.18,21146,19853.00,16374.82,52691,0.00,0.00,0,No,False,False,False,True,False
3,355,1716.05,55464,5769.17,4053.13,42137,0.00,0.00,0,No,False,False,False,True,False
4,354,253129.93,30759,1328499.49,1581629.42,26864,2713220.48,2460090.55,0,No,True,False,False,False,False


In [ ]:
data['source_amount'] = (data['Initial_Origin_Balance'] - data['Final_Origin_Balance'])
data['destination_amount'] = (data['Final_Destination_Balance'] - data['Initial_Destination_Balance'])

In [10]:
test_data['source_amount'] = (test_data['Initial_Origin_Balance'] - test_data['Final_Origin_Balance'])
test_data['destination_amount'] = (test_data['Final_Destination_Balance'] - test_data['Initial_Destination_Balance'])

In [ ]:

money_columns = ['Amount', 'source_amount', 'destination_amount']

In [ ]:
data[(data['source_amount'].abs() == data['destination_amount'].abs()) & (data['source_amount'] != data['Amount'].abs())][money_columns].head()

In [ ]:
swappables = (data['source_amount'] <= 0) & (data['destination_amount'] <= 0)
data.loc[swappables, 'source_amount'], data.loc[swappables, 'destination_amount'] = data.loc[swappables, 'destination_amount'], data.loc[swappables, 'source_amount']

In [11]:
test_swappables = (test_data['source_amount'] <= 0) & (test_data['destination_amount'] <= 0)
test_data.loc[test_swappables, 'source_amount'], test_data.loc[test_swappables, 'destination_amount'] = test_data.loc[test_swappables, 'destination_amount'], test_data.loc[test_swappables, 'source_amount']

In [ ]:
data['time_of_day'] = data['Time']%24
data.groupby('Fraud')['time_of_day'].value_counts()


In [ ]:
data.groupby('Fraud')['Time'].value_counts()


In [12]:
columns_to_drop = [
    'Origin_ID', 'Destination_ID', 'Initial_Origin_Balance', 
    'Initial_Destination_Balance', 'Expected_Fraud', 
    'source_amount', 'destination_amount', 'TransactionAmount', 'time_of_day', 'Time'
]

# Drop columns if they exist
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns])
test_data = test_data.drop(columns=[col for col in columns_to_drop if col in test_data.columns])

data.head()

,Amount,Final_Origin_Balance,Final_Destination_Balance,Fraud,Transaction_Type_CASH_IN,Transaction_Type_CASH_OUT,Transaction_Type_DEBIT,Transaction_Type_PAYMENT,Transaction_Type_TRANSFER
0,183806.32,0.00,566378.51,0,False,True,False,False,False
1,521.37,0.00,0.00,0,False,False,False,True,False
2,3478.18,16374.82,0.00,0,False,False,False,True,False
3,1716.05,4053.13,0.00,0,False,False,False,True,False
4,253129.93,1581629.42,2460090.55,0,True,False,False,False,False


In [ ]:
data.head()

In [ ]:
data.to_csv('cleaned_data.csv', index=False)

In [13]:
test_data.to_csv('cleaned_test_data.csv', index=False)